In [20]:
# Uploading libraries
import ast
import pandas as pd
import seaborn as sns
import numpy as np
from datasets import load_dataset
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from adjustText import adjust_text


# Uploading dataset
df_2019_q1 = pd.read_csv('Original_data_Q1_2019_Q1_2020/Divvy_Trips_2019_Q1.csv')



In [21]:
df_2019_q1.sample(5)

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
174876,21952323,2019-02-22 07:53:55,2019-02-22 08:02:53,1818,538.0,91,Clinton St & Washington Blvd,197,Michigan Ave & Madison St,Subscriber,Male,1969.0
8010,21752119,2019-01-03 14:08:22,2019-01-03 14:19:30,3632,668.0,43,Michigan Ave & Washington St,66,Clinton St & Lake St,Subscriber,Male,1992.0
125562,21891247,2019-02-06 20:01:37,2019-02-06 20:15:20,332,823.0,174,Canal St & Madison St,180,Ritchie Ct & Banks St,Subscriber,Male,1988.0
277417,22076219,2019-03-18 12:09:27,2019-03-18 12:22:09,4357,762.0,140,Dearborn Pkwy & Delaware Pl,40,LaSalle St & Adams St,Subscriber,Male,1984.0
96042,21854691,2019-01-24 15:42:19,2019-01-24 15:55:08,5903,769.0,43,Michigan Ave & Washington St,74,Kingsbury St & Erie St,Subscriber,Male,1993.0


In [22]:
df_2019_q1['usertype'] = df_2019_q1['usertype'].replace('Customer', 'casual')
df_2019_q1['usertype'] = df_2019_q1['usertype'].replace('Subscriber', 'member')

In [23]:
df_2019_q1['usertype'].value_counts()

usertype
member    341906
casual     23163
Name: count, dtype: int64

In [24]:
columns_to_drop = ["tripduration", "from_station_id", "from_station_name", "to_station_id", "to_station_name", "gender", "birthyear", "bikeid"]
df_clean_2019_q1 = df_2019_q1.drop(columns=columns_to_drop)

In [25]:
df_clean_2019_q1.sample(5)

,trip_id,start_time,end_time,usertype
180269,21958883,2019-02-23 15:57:59,2019-02-23 16:05:23,member
278866,22078021,2019-03-18 16:29:50,2019-03-18 16:39:38,member
47085,21796478,2019-01-10 14:52:20,2019-01-10 15:01:46,member
109082,21871844,2019-02-03 17:50:12,2019-02-03 17:59:00,member
186812,21967049,2019-02-26 08:18:48,2019-02-26 08:21:52,member


In [26]:
# Formatted columns start_time & end_time from Series to date&time then created new column where calculated trip duration

df_clean_2019_q1['start_time'] = pd.to_datetime(df_clean_2019_q1['start_time'], format='%Y-%m-%d %H:%M:%S')
df_clean_2019_q1['end_time'] = pd.to_datetime(df_clean_2019_q1['end_time'], format='%Y-%m-%d %H:%M:%S')


df_clean_2019_q1['ride_length_seconds'] = (df_clean_2019_q1['end_time'] - df_clean_2019_q1['start_time']).dt.total_seconds()

df_clean_2019_q1['ride_length_seconds'] = df_clean_2019_q1['ride_length_seconds'].astype(int)


def seconds_to_hhmmss(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


df_clean_2019_q1['ride_length'] = df_clean_2019_q1['ride_length_seconds'].apply(seconds_to_hhmmss)


df_clean_2019_q1.sample(10)


,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length
114614,21878295,2019-02-04 19:16:40,2019-02-04 19:40:53,member,1453,00:24:13
226248,22015474,2019-03-08 14:11:58,2019-03-08 14:26:12,member,854,00:14:14
109078,21871840,2019-02-03 17:49:23,2019-02-03 17:53:41,member,258,00:04:18
133896,21901967,2019-02-09 15:26:45,2019-02-09 15:33:46,member,421,00:07:01
280118,22079386,2019-03-18 17:36:33,2019-03-18 17:52:22,member,949,00:15:49
238480,22030267,2019-03-11 18:19:22,2019-03-11 18:24:33,member,311,00:05:11
87024,21843476,2019-01-21 08:54:30,2019-01-21 09:06:12,member,702,00:11:42
285090,22085125,2019-03-19 12:38:04,2019-03-19 12:41:25,member,201,00:03:21
143897,21914745,2019-02-13 12:32:14,2019-02-13 13:01:22,member,1748,00:29:08
219547,22007112,2019-03-07 07:04:10,2019-03-07 07:15:07,member,657,00:10:57


In [27]:
# Extract the day of the week and create a new column
df_clean_2019_q1['day_of_week'] = df_clean_2019_q1['start_time'].dt.day_name()

# Print the first few rows to verify
df_clean_2019_q1.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length,day_of_week
102331,21863461,2019-01-29 14:59:28,2019-01-29 15:09:11,member,583,00:09:43,Tuesday
76667,21830939,2019-01-17 09:50:27,2019-01-17 09:58:41,member,494,00:08:14,Thursday
299567,22102209,2019-03-21 17:07:33,2019-03-21 17:17:52,member,619,00:10:19,Thursday
162652,21937721,2019-02-19 10:44:52,2019-02-19 11:10:11,member,1519,00:25:19,Tuesday
155389,21928582,2019-02-15 19:35:45,2019-02-15 19:49:01,member,796,00:13:16,Friday
75704,21829849,2019-01-17 08:02:17,2019-01-17 08:17:30,member,913,00:15:13,Thursday
163363,21938730,2019-02-19 15:19:58,2019-02-19 15:26:54,member,416,00:06:56,Tuesday
282637,22082169,2019-03-19 07:31:06,2019-03-19 07:38:45,member,459,00:07:39,Tuesday
184892,21964893,2019-02-25 17:36:02,2019-02-25 17:39:19,member,197,00:03:17,Monday
230139,22019911,2019-03-09 12:23:24,2019-03-09 12:32:11,member,527,00:08:47,Saturday


In [28]:
df_clean_2019_q1.to_csv('Cleaned_2019_Q1.csv', index=False)